The `predict_event` function process a fridge's CSV data file, employ a pre-trained Random Forest model to generate predictions, and manage any errors that could occur during this operation. In the context of the predictions, the output is binary, with "1" representing an ON/OFF event and "0" indicating a KEEP ALIVE event.

In [4]:
import pandas as pd
from joblib import load

def predict_event(csv_file):
    try:
        
        df = pd.read_csv(csv_file)

        # Ensure the 'timestamp' column exists in the DataFrame
        if 'timestamp' not in df.columns:
            raise ValueError("The input CSV file must have a 'timestamp' column.")

        # Convert and preprocess the data as needed
        df['timestamp'] = pd.to_datetime(df['timestamp'], format="%Y-%m-%d %H:%M:%S.%f", errors='coerce')
        df['timestamp'] = df['timestamp'].dt.strftime("%Y-%m-%d %H:%M:%S")
        df['timestamp'] = pd.to_datetime(df['timestamp'], format="%Y-%m-%d %H:%M:%S")
        df['hour'] = df['timestamp'].dt.hour
        df['minute'] = df['timestamp'].dt.minute
        df.drop('timestamp', axis=1, inplace=True)
        df = df[['active_power', 'current', 'transient_1', 'transient_2', 'transient_3', 'transient_4', 'transient_5', 'transient_7',
                 'peak_1', 'peak_2', 'peak_3', 'peak_4', 'peak_5', 'peak_6', 'peak_7', 'peak_8', 'peak_9', 'peak_10', 'hour']]

        columns_to_fill = {
            'current': 'mean',
            'active_power': 'mean',
        }
        
        for column, method in columns_to_fill.items():
            if method == 'mean':
                mean_value = df[column].mean()
                df[column].fillna(mean_value, inplace=True)
        
        transient_columns = ['transient_1', 'transient_2', 'transient_3', 'transient_4', 'transient_5', 'transient_7']
        peak_columns = ['peak_1', 'peak_2', 'peak_3', 'peak_4', 'peak_5', 'peak_6', 'peak_7', 'peak_8', 'peak_9', 'peak_10']

        df[transient_columns] = df[transient_columns].interpolate(method='linear', axis=0)
        df[peak_columns] = df[peak_columns].interpolate(method='linear', axis=0)

        # Drop rows with NaN values in the input data
        df.dropna(inplace=True)

        # Reset the index of the DataFrame
        df.reset_index(drop=True, inplace=True)
        
        # Load the trained model
        model = load('fridge_rf_model.joblib')

        # Make predictions
        predictions = model.predict(df)


        return predictions

    except FileNotFoundError:
        return "Error: The specified CSV file was not found."

    except ValueError as ve:
        return str(ve)

    except Exception as e:
        return f"An error occurred: {str(e)}"


In [5]:
# Prediction:

csv_file = 'random.csv' 
result = predict_event(csv_file)

print(result)



[1 0 0 1 0 0 1 0 1 1]
